In [218]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torch.nn.functional as F
import torch.utils as utils
import torchvision
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF
import torchvision.datasets as dsets
from torch.utils.data import Dataset,DataLoader
from torchvision import models

import cv2

import matplotlib.pyplot as plt

import os
os.chdir("/projects/2019연구학점제/")
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import time
import copy
import random as rd

device = torch.device('cuda')

<h1>Data</h1>

In [5]:
#Custom pytorch Dataset for Adobe 240fps dataset in hdd
rd.seed(0)

#Train with Adobe 240fps dataset
class train_set(Dataset):
    def __init__(self):
        self.dir = "../data/adobe240fps/"
        self.folder_list = os.listdir(self.dir)
        self.seq_len = 12
        
        self.data_paths = []
        for folder_dir in self.folder_list:
            files = sorted([x for x in os.listdir(self.dir + folder_dir + "/240/" + folder_dir) if ".jpg" in x])
            files = files[:len(files) - len(files)%self.seq_len]
            
            for seq_ind in range(int(len(files)/self.seq_len)):
                paths = []
                for ind in range(self.seq_len):
                    paths.append(self.dir + folder_dir + "/240/" + folder_dir + "/" + files[seq_ind * self.seq_len + ind])
                self.data_paths.append(copy.deepcopy(paths))
        self.len = len(self.data_paths)

        self.transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize((128.0, 128.0, 128.0), (128.0, 128.0, 128.0))
        ])
        
    def __len__(self):
        return self.len
        
    def __getitem__(self, idx):
        for ind, img_dir in enumerate(self.data_paths[idx]):
            img = cv2.imread(img_dir)
            if img is None:
                print("Image load failed: %s" %img_dir)
                continue
            if ind:
                imgs = torch.cat((imgs, self.transform(img).unsqueeze(0)))
            else:
                imgs = self.transform(img).unsqueeze(0)
        if imgs.shape[2] > imgs.shape[3]:
            imgs = imgs.transpose(2,3)
        return imgs.transpose(0,1)

In [219]:
train_loader = DataLoader(train_set(), batch_size = 16, shuffle = True, num_workers = 0)

In [220]:
%%time
for video in train_loader:
    print(video.shape)
    break

torch.Size([16, 3, 12, 224, 224])
CPU times: user 7.71 s, sys: 0 ns, total: 7.71 s
Wall time: 3.75 s


<h1>Load data on Memory</h1>

In [6]:
class train_data:
    def __init__(self, batch_size = 4):
        self.batch_size = batch_size
        self.loader = DataLoader(train_set(), batch_size = self.batch_size, shuffle = True, num_workers = 0)
        self.batch_size = self.loader 
        self.len = len(self.loader)
        self.data = []
        print("Total length: %d" %self.len)
        for ind, video in enumerate(self.loader):
            self.data.append(copy.deepcopy(video))
            if not ind%100:
                print(ind)
        self.shuffle()
        self.order = list(range(self.len))
        
    def __len__(self):
        return self.len
    
    def __getitem__(self, idx):
        for ind in range(self.batch_size):
            if ind:
                videos = torch.cat((videos, data[idx * self.batch_size + ind].unsqueeze(0)))
            else:
                videos = data[idx * self.batch_size].unsqueeze(0)
        return videos
    
    def shuffle(self):
        rd.shuffle(self.data)

<h1>Test Set</h1>

In [ ]:
#Test With test set of middlburry
class test_set_middleburry(Dataset):
    def __init__(self):
        self.dir = "../"


<h1>Model</h1>

In [ ]:
class sample_net1(nn.Module):
    def __init__(self):
        super(sample_net1, self)__init__()
        
        
        
        
        
        

In [215]:
class testnet1(nn.Module):
    def __init__(self):
        super(testnet1, self).__init__()
        
        ###Encoder
        self.enc_l1 = nn.Sequential(
            nn.BatchNorm3d(3),
            nn.LeakyReLU(),
            nn.Conv3d(3, 3, kernel_size = (1, 1, 1)),
            
            nn.BatchNorm3d(3),
            nn.LeakyReLU(),
            nn.Conv3d(3, 3, kernel_size = (3, 1, 1), padding = (1, 0, 0)),
            nn.Conv3d(3, 3, kernel_size = (1, 3, 3), padding = (0, 3, 3), dilation = (1, 3, 3))
        )
        self.enc_tr1 = nn.Conv3d(6, 6, kernel_size = (1, 1, 1), stride = (1, 2, 2))
        
        self.enc_l2 = nn.Sequential(
            nn.BatchNorm3d(6),
            nn.LeakyReLU(),
            nn.Conv3d(6, 3, kernel_size = (1, 1, 1)),
            
            nn.BatchNorm3d(3),
            nn.LeakyReLU(),
            nn.Conv3d(3, 3, kernel_size = (3, 1, 1), padding = (1, 0, 0)),
            nn.Conv3d(3, 3, kernel_size = (1, 3, 3), padding = (0, 3, 3), dilation = (1, 3, 3))
        )
        self.enc_tr2 = nn.Conv3d(9, 9, kernel_size = (1, 1, 1), stride = (1, 2, 2))
        
        self.enc_l3 = nn.Sequential(
            nn.BatchNorm3d(9),
            nn.LeakyReLU(),
            nn.Conv3d(9, 3, kernel_size = (1, 1, 1)),
            
            nn.BatchNorm3d(3),
            nn.LeakyReLU(),
            nn.Conv3d(3, 3, kernel_size = (3, 1, 1), padding = (1, 0, 0)),
            nn.Conv3d(3, 3, kernel_size = (1, 3, 3), padding = (0, 3, 3), dilation = (1, 3, 3))
        )
        self.enc_tr3 = nn.Conv3d(12, 12, kernel_size = (1, 1, 1), stride = (1, 2, 2))
        
        self.enc_l4 = nn.Sequential(
            nn.BatchNorm3d(12),
            nn.LeakyReLU(),
            nn.Conv3d(12, 3, kernel_size = (1, 1, 1)),
            
            nn.BatchNorm3d(3),
            nn.LeakyReLU(),
            nn.Conv3d(3, 3, kernel_size = (3, 1, 1), padding = (1, 0, 0)),
            nn.Conv3d(3, 3, kernel_size = (1, 3, 3), padding = (0, 3, 3), dilation = (1, 3, 3))
        )
        self.enc_tr4 = nn.Conv3d(15, 15, kernel_size = (1, 1, 1), stride = (1, 2, 2))
        
        self.enc_l5 = nn.Sequential(
            nn.BatchNorm3d(15),
            nn.LeakyReLU(),
            nn.Conv3d(15, 3, kernel_size = (1, 1, 1)),
            
            nn.BatchNorm3d(3),
            nn.LeakyReLU(),
            nn.Conv3d(3, 3, kernel_size = (3, 1, 1), padding = (1, 0, 0)),
            nn.Conv3d(3, 3, kernel_size = (1, 3, 3), padding = (0, 3, 3), dilation = (1, 3, 3))
        )
        self.enc_tr5 = nn.Conv3d(18, 18, kernel_size = (1, 1, 1), stride = (1, 2, 2))
        
        self.enc_l6 = nn.Sequential(
            nn.BatchNorm3d(18),
            nn.LeakyReLU(),
            nn.Conv3d(18, 3, kernel_size = (1, 1, 1)),
            
            nn.BatchNorm3d(3),
            nn.LeakyReLU(),
            nn.Conv3d(3, 3, kernel_size = (3, 1, 1), padding = (1, 0, 0)),
            nn.Conv3d(3, 3, kernel_size = (1, 3, 3), padding = (0, 3, 3), dilation = (1, 3, 3))
        )
        self.enc_tr6 = nn.Conv3d(21, 21, kernel_size = (1, 1, 1), stride = (1, 2, 2))
        
        self.enc_l7 = nn.Sequential(
            nn.BatchNorm3d(21),
            nn.LeakyReLU(),
            nn.Conv3d(21, 3, kernel_size = (1, 1, 1)),
            
            nn.BatchNorm3d(3),
            nn.LeakyReLU(),
            nn.Conv3d(3, 3, kernel_size = (3, 1, 1), padding = (1, 0, 0)),
            nn.Conv3d(3, 3, kernel_size = (1, 3, 3), padding = (0, 3, 3), dilation = (1, 3, 3))
        )
        self.enc_tr7 = nn.Conv3d(24, 24, kernel_size = (1, 1, 1), stride = (1, 2, 2))
        
        self.enc_l8 = nn.Sequential(
            nn.BatchNorm3d(24),
            nn.LeakyReLU(),
            nn.Conv3d(24, 3, kernel_size = (1, 1, 1)),
            
            nn.BatchNorm3d(3),
            nn.LeakyReLU(),
            nn.Conv3d(3, 3, kernel_size = (3, 1, 1), padding = (1, 0, 0)),
            nn.Conv3d(3, 3, kernel_size = (1, 3, 3), padding = (0, 3, 3), dilation = (1, 3, 3))
        )
        self.enc_tr8 = nn.Conv3d(27, 27, kernel_size = (1, 1, 1), stride = (1, 2, 2))
        
        self.enc_l9 = nn.Sequential(
            nn.BatchNorm1d(324),
            nn.LeakyReLU(),
            nn.Linear(324, 2 * , kernel_size = (1, 1, 1)),
        )
        

    def forward(self, x):
        x = self.enc_tr1(torch.cat((x, self.enc_l1(x)), 1))
        x = self.enc_tr2(torch.cat((x, self.enc_l2(x)), 1))
        x = self.enc_tr3(torch.cat((x, self.enc_l3(x)), 1))
        x = self.enc_tr4(torch.cat((x, self.enc_l4(x)), 1))
        x = self.enc_tr5(torch.cat((x, self.enc_l5(x)), 1))
        x = self.enc_tr6(torch.cat((x, self.enc_l6(x)), 1))
        x = self.enc_tr7(torch.cat((x, self.enc_l7(x)), 1))
        x = self.enc_tr8(torch.cat((x, self.enc_l8(x)), 1))
        return x.view(x.shape[0], -1)

In [216]:
sampler_net = testnet1().cuda()
sample_net = 

In [217]:
%%time
print(video.shape)
print(sampler_net(video.cuda()).shape)

torch.Size([16, 3, 12, 224, 224])
torch.Size([16, 324])
CPU times: user 27.5 ms, sys: 647 µs, total: 28.1 ms
Wall time: 27.7 ms
